<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/sparsity_4_optim_pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import DataLoader, Subset
import torchvision.datasets as datasets
from torchvision import transforms
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(1234)
np.random.seed(1234)

In [3]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [4]:
# class_inds 이거는 그냥 위에있는거를 list 로 만들어준 형태임 
class_inds = [torch.where(mnist_trainset.targets == class_idx)[0]
              for class_idx in mnist_trainset.class_to_idx.values()]

In [5]:
train_dataloaders = [
                     DataLoader(dataset=Subset(mnist_trainset, inds),
                                batch_size=50,
                                shuffle=True,
                                drop_last=False
                     )
                     for inds in class_inds
]

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

print("Training dataset size: ", len(mnist_trainset))
print("Testing dataset size: ",  len(mnist_testset))

Training dataset size:  60000
Testing dataset size:  10000


In [6]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [7]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [8]:
def sparsity_calculator(final_spareness):
    sparseness_list = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return sparseness_list

In [9]:
def model_factory(optimizer_name):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    # modify this section for later use 
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparseness_list = sparsity_trainer(optimizer=my_optimizer, model=my_model)
    # ************* modify this section for later use *************
    file_saver = open(f"sparsity_4_optim_single_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparseness_list)+'\n\n')
    file_saver.close()

    # ************* modify this section for later use *************
    if optimizer_name == 'Adadelta':
        !cp sparsity_4_optim_single_Adadelta.txt /content/drive/MyDrive
    
    elif optimizer_name == 'Adagrad':
        !cp sparsity_4_optim_single_Adagrad.txt /content/drive/MyDrive

    elif optimizer_name == 'SGD':
        !cp sparsity_4_optim_single_SGD.txt /content/drive/MyDrive

    elif optimizer_name == 'Adam':
        !cp sparsity_4_optim_single_Adam.txt /content/drive/MyDrive
    # ************* modify this section for later use *************

    else:
        print("ERROR")

In [10]:
def sparsity_trainer(optimizer, model):
    criterion = torch.nn.CrossEntropyLoss()
    no_epochs = 30
    test_acc   = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        print(f"epoch {epoch} started")

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        # ************* modify this section for later use *************

        # TRAINING 
        model.train()
        iterators = list(map(iter, train_dataloaders))   
        while iterators:    # This part is same as for loop 
            iterator = np.random.choice(iterators)
            try:
                image, label = next(iterator)   
                image, label = image.to(device), label.to(device)

                optimizer.zero_grad()

                pred = model(image)

                loss = criterion(pred, label)

                loss.backward()
                optimizer.step()
                
            except StopIteration:
                iterators.remove(iterator)

        # TESTING
        model.eval()
        total = 0
        for itr, (image, label) in enumerate(test_dataloader):
            image, label = image.to(device), label.to(device)

            pred = model(image)

            loss = criterion(pred, label)

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if label[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        # ***************** sparsity calculation ***************** # 

        accuracy = total / len(mnist_testset)
        # append accuracy here
        test_acc.append(accuracy)

        print('\nEpoch: {}/{}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12 = sparsity_calculator(final_spareness_12)

    average_sparsity = list()
    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    print("average_sparsity:", average_sparsity)

    return test_acc, average_sparsity

# Adadelta

In [11]:
model_factory('Adadelta')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)
epoch 0 started

Epoch: 1/30, Test Accuracy: 0.57570000
epoch 1 started

Epoch: 2/30, Test Accuracy: 0.65080000
epoch 2 started

Epoch: 3/30, Test Accuracy: 0.82620000
epoch 3 started

Epoch: 4/30, Test Accuracy: 0.79340000
epoch 4 started

Epoch: 5/30, Test Accuracy: 0.85140000
epoch 5 started

Epoch: 6/30, Test Accuracy: 0.87090000
epoch 6 started

Epoch: 7/30, Test Accuracy: 0.88610000
epoch 7 started

Epoch: 8/30, Test Accuracy: 0.88800000
epoch 8 started

Epoch: 9/30, Test Accuracy: 0.92150000
epoch 9 started

Epoch: 10/30, Test Accuracy: 0.91440000
epoch 10 started

Epoch: 11/30, Test Accuracy: 0.89630000
epoch 11 started

Epoch: 12/30, Test Accuracy: 0.93980000
epoch 12 started

Epoch: 13/30, Test Accu

# Adagrad

In [12]:
model_factory('Adagrad')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    weight_decay: 0
)
epoch 0 started

Epoch: 1/30, Test Accuracy: 0.78540000
epoch 1 started

Epoch: 2/30, Test Accuracy: 0.93130000
epoch 2 started

Epoch: 3/30, Test Accuracy: 0.94420000
epoch 3 started

Epoch: 4/30, Test Accuracy: 0.94550000
epoch 4 started

Epoch: 5/30, Test Accuracy: 0.94460000
epoch 5 started

Epoch: 6/30, Test Accuracy: 0.95810000
epoch 6 started

Epoch: 7/30, Test Accuracy: 0.95760000
epoch 7 started

Epoch: 8/30, Test Accuracy: 0.96660000
epoch 8 started

Epoch: 9/30, Test Accuracy: 0.96180000
epoch 9 started

Epoch: 10/30, Test Accuracy: 0.96850000
epoch 10 started

Epoch: 11/30, Test Accuracy: 0.96870000
epoch 11 started

Epoch: 12/30, Test Accuracy: 0.97200000
epoch 

# SGD

In [13]:
model_factory('SGD')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)
epoch 0 started

Epoch: 1/30, Test Accuracy: 0.71240000
epoch 1 started

Epoch: 2/30, Test Accuracy: 0.80000000
epoch 2 started

Epoch: 3/30, Test Accuracy: 0.81740000
epoch 3 started

Epoch: 4/30, Test Accuracy: 0.85630000
epoch 4 started

Epoch: 5/30, Test Accuracy: 0.82890000
epoch 5 started

Epoch: 6/30, Test Accuracy: 0.88940000
epoch 6 started

Epoch: 7/30, Test Accuracy: 0.91720000
epoch 7 started

Epoch: 8/30, Test Accuracy: 0.90490000
epoch 8 started

Epoch: 9/30, Test Accuracy: 0.92690000
epoch 9 started

Epoch: 10/30, Test Accuracy: 0.92610000
epoch 10 started

Epoch: 11/30, Test Accuracy: 0.91890000
epoch 11 started

Epoch: 12/30, Test Accuracy: 0.92530000
epoch 12 started

Epo

# Adam

In [14]:
model_factory('Adam')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
epoch 0 started

Epoch: 1/30, Test Accuracy: 0.77460000
epoch 1 started

Epoch: 2/30, Test Accuracy: 0.83920000
epoch 2 started

Epoch: 3/30, Test Accuracy: 0.88340000
epoch 3 started

Epoch: 4/30, Test Accuracy: 0.82650000
epoch 4 started

Epoch: 5/30, Test Accuracy: 0.89370000
epoch 5 started

Epoch: 6/30, Test Accuracy: 0.89660000
epoch 6 started

Epoch: 7/30, Test Accuracy: 0.88640000
epoch 7 started

Epoch: 8/30, Test Accuracy: 0.92640000
epoch 8 started

Epoch: 9/30, Test Accuracy: 0.93700000
epoch 9 started

Epoch: 10/30, Test Accuracy: 0.95220000
epoch 10 started

Epoch: 11/30, Test Accuracy: 0.93940000
epoch 11 started

Epoch: 12/30, Test Accuracy: 0.94280000
epoch 12 star